In [1]:
#import streamlit as st 
import requests, json
import pandas as pd
import numpy as np
import cv2
#import pydeck as pdk

# 실시간 위치 좌표 얻기
---
- 출처: https://parkgihyeon.github.io/project/geocoding-api/

In [84]:
# 현재위치 좌표 얻기
import requests, json

def current_location():
    here_req = requests.get("http://www.geoplugin.net/json.gp")

    if (here_req.status_code != 200):
        print("현재좌표를 불러올 수 없음")
    else:
        location = json.loads(here_req.text)
        crd = {"lat": str(location["geoplugin_latitude"]), "lng": str(location["geoplugin_longitude"])}

    return crd

crd = current_location()
print(crd)

{'lat': '35.8723', 'lng': '128.5924'}


In [5]:
# 가입 없이 좌표->주소 변환
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

address = geocoding_reverse('35.8908017,128.6506577')
print(address)

입석로, 동구, 대구, 41143, 대한민국


In [16]:
# 가입 없이 주소->좌표 변환 ###
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

crd = geocoding("대구 동도초등학교")
print(crd['lat'])
print(crd['lng'])

gps = pd.DataFrame( [[crd['lat'],crd['lng']]], columns=['위도','경도'])
gps

35.85665565
128.62401223992515


,위도,경도
0,35.85665565,128.62401223992515


# 장소 마크 표시 및 개수 세기
---- 
참고:https://haries.tistory.com/6

# streamlit yolov5 
---
참고 : https://github.com/xugaoxiang/yolov5-streamlit

# streamlit 드론 gps
---
참고: https://github.com/greattoe/ros_tutorial_kr/blob/master/ros1_tutorial/rospy/bebop2/bb2_5_mark_bebop2_on_web.md

# 도로 CCTV 송출
---
참고: https://parkgihyeon.github.io/project/cctv-api/#2%EF%B8%8F%E2%83%A3-cctv-api%EB%A5%BC-%ED%98%B8%EC%B6%9C%ED%95%98%EB%8A%94-%EB%B0%A9%EB%B2%95

# 수정코드
---

In [87]:
# 현재위치 좌표 얻기
import requests, json
import pandas as pd

def current_location():
    here_req = requests.get("http://www.geoplugin.net/json.gp")

    if (here_req.status_code != 200):
        print("현재좌표를 불러올 수 없음")
    else:
        location = json.loads(here_req.text)
        crd = {float(location["geoplugin_latitude"]), float(location["geoplugin_longitude"])}
        crd = list(crd)
        gps = pd.DataFrame( [[crd[1],crd[0]]], columns=['위도','경도'])
    
    return gps

In [88]:
gps = current_location()
gps

,위도,경도
0,35.8723,128.5924


In [89]:
# gps_all(기존) 불러오기
gps_all = pd.read_csv('gps_all.csv')

# gps_all(기존), gps(추가 갱신) 데이터프레임 결합 
gps_all = pd.concat([gps_all,gps]).reset_index()
gps_all = gps_all.drop('index',axis=1)

# 중복 위치 제거
gps_all = gps_all.drop_duplicates(['위도','경도'])
# gps_all,len(gps_all,'개')

# 추가 위치정보 저장된 데이터프레임 저장
gps_all.to_csv('gps_all.csv',index = False)

In [90]:
gps_all = pd.read_csv('gps_all.csv')
gps_all

,위도,경도
0,35.8124,128.5125
1,35.8825,128.5726
2,35.8326,128.5327
3,35.8627,128.5428
4,35.8428,128.5529
5,35.8723,128.5924


In [91]:
# 가입 없이 좌표->주소 변환
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

address = geocoding_reverse('35.8723, 128.5924')
print(address)

경상감영1길, 중구, 대구, 41934, 대한민국


In [92]:
#for i in range(7):
lat = gps_all['위도'][0]
lng = gps_all['경도'][0]
print(f'{lat},{lng}')

35.8124,128.5125


In [117]:
option='중구'

In [120]:
# 가입 없이 좌표->주소 변환
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

address_list = []
for i in range(len(gps_all)):
#    if option in gps_all.loc[i]:
        lat = gps_all['위도'][i]
        lng = gps_all['경도'][i]
        address = geocoding_reverse(f'{lat}, {lng}')
        address_list.append(address)

In [121]:
df = pd.DataFrame(address_list, columns=['주소','위치정보(위도,경도)'])

df

,주소,"위치정보(위도,경도)"
0,"대구유천초등학교, 달서대로, 유천동, 달서구, 대구, 42769, 대한민국","(35.812507, 128.512028)"
1,"달서로, 비산6동, 비산동, 서구, 대구, 41782, 대한민국","(35.88249340995197, 128.57261594921147)"
2,"구마로14남길, 월성센트로파크아파트, 월성동, 달서구, 대구, 42730, 대한민국","(35.832596088998464, 128.53256479938742)"
3,"27, 평리로35길, 서대구공단, 서구, 대구, 41844, 대한민국","(35.862623049999996, 128.542901)"
4,"야외음악당로, 송현2동, 성당동, 달서구, 대구, 42671, 대한민국","(35.84280009420493, 128.55287741436584)"
5,"경상감영1길, 중구, 대구, 41934, 대한민국","(35.87236884687725, 128.592956121765)"


# 정규식으로 구별 현황 파악
---

In [196]:
option = '대구 전체'
select_list =[]

In [156]:
gps_all = pd.read_csv('gps_all.csv')
len(gps_all)

6

In [197]:
address_list = []
for i in range(len(gps_all)):                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
    lat = gps_all['위도'][i]
    lng = gps_all['경도'][i]
    address = geocoding_reverse(f'{lat}, {lng}')
    
    # 카테고리 선택 
    if option =='대구 전체':
        address_list.append(address)
    elif option in address[0]:
        address_list.append(address)

df = pd.DataFrame(address_list, columns=['주소','위치정보(위도,경도)'])


In [198]:
df.loc[0][1][0]

35.812507

In [199]:
df_map = pd.DataFrame(columns=['주소','위도','경도'])
for i in range(len(df)):
    df_map.loc[i] = [df.loc[i]['주소'],df.loc[i][1][0],df.loc[i][1][1]]

In [200]:
df_map

,주소,위도,경도
0,"대구유천초등학교, 달서대로, 유천동, 달서구, 대구, 42769, 대한민국",35.812507,128.512028
1,"달서로, 비산6동, 비산동, 서구, 대구, 41782, 대한민국",35.882493,128.572616
2,"구마로14남길, 월성센트로파크아파트, 월성동, 달서구, 대구, 42730, 대한민국",35.832596,128.532565
3,"27, 평리로35길, 서대구공단, 서구, 대구, 41844, 대한민국",35.862623,128.542901
4,"야외음악당로, 송현2동, 성당동, 달서구, 대구, 42671, 대한민국",35.842800,128.552877
5,"경상감영1길, 중구, 대구, 41934, 대한민국",35.872369,128.592956


# 이미지업로드
---

In [2]:
!pip install items

Defaulting to user installation because normal site-packages is not writeable


In [6]:
# 이미지 위도, 경도
from PIL import Image
from PIL.ExifTags import TAGS
import pandas as pd
import numpy as np

f = './test3.jpg'
image = Image.open(f)
info = image._getexif() # 이미지의 정보 추출
image.close()
taglabel = {}
for tag, value in info.items():
    decoded = TAGS.get(tag, tag)
    taglabel[decoded] = value
exifGPS = taglabel['GPSInfo']
exifGPS

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-1: ordinal not in range(256)